In [1]:
import numpy as np
import pandas as pd

In [2]:
df0 = pd.read_csv("DATA/DMC_2017_task/train_clean.csv")
del df0["Unnamed: 0"]
df0.shape

(2756003, 23)

In [12]:
df0.head(100)

,day,pid,click,basket,order,revenue,lineID,number,group,unit,...,adFlag,availability,competitorPrice,price,manufacturer,genericProduct,salesIndex,category,rrp,Content
0,1,6570,1,0,0,0.00,0,0.0,2FOI,ML,...,0.0,0.333333,0.055180,0.044533,0.238274,0.0,0.000000,0.470588,0.044900,0.002261
1,1,14922,0,1,0,0.00,1,0.0,1COJ0FIK,ST,...,1.0,0.000000,0.032390,0.023045,0.015947,1.0,0.000000,0.159314,0.046283,0.002261
2,1,16382,0,1,0,0.00,2,0.0,22OI7,ML,...,0.0,0.000000,0.055822,0.042342,0.037523,0.0,1.000000,0.095588,0.045468,0.004534
3,1,1145,0,0,1,6.55,3,1.0,18OZ00IS,G,...,1.0,0.000000,0.024906,0.017238,0.047842,0.0,0.000000,0.058824,0.022820,0.002716
4,1,3394,0,0,1,4.14,4,1.0,20OI0,ST,...,0.0,0.000000,0.016592,0.010876,0.083490,0.0,1.000000,0.031863,0.019906,0.002261
5,1,3661,0,0,1,10.03,5,1.0,13OX06,ML,...,0.0,0.000000,0.051627,0.026424,0.083490,0.0,0.923077,0.308824,0.053174,0.045444
6,1,3856,0,0,1,3.58,6,1.0,13OK0FOK,G,...,1.0,0.000000,0.011452,0.009398,0.077861,0.0,0.000000,0.218137,0.013707,0.000898
7,1,16963,1,0,0,0.00,7,0.0,22OI3,ML,...,0.0,0.000000,0.033183,0.023045,0.082552,0.0,1.000000,0.017157,0.028526,0.004534
8,1,14560,1,0,0,0.00,8,0.0,22OI3,ML,...,0.0,0.000000,0.040969,0.031730,0.834897,0.0,1.000000,0.017157,0.034873,0.002261
9,1,4853,1,0,0,0.00,9,0.0,13OX04OZ,G,...,1.0,0.000000,0.034468,0.023045,0.552533,0.0,0.000000,0.218137,0.035021,0.004534


In [4]:
df = df0[df0.day < 62]

In [5]:
df1 = df.drop(['day', 'pid', 'lineID', "click", "basket", "revenue", "number", "group", "pharmForm", "unit", 'campaignIndex'], axis=1)
print(df1.shape)
print(df1["order"].value_counts())

(1746818, 12)
0    1285304
1     461514
Name: order, dtype: int64


In [6]:
from keras.utils import np_utils

X = np.array(df1.drop(["order"], axis=1))
y = np_utils.to_categorical(np.array(df1[["order"]]), 2) 

Using TensorFlow backend.


In [7]:
X_train = X[0:1397455, :]
y_train = y[0:1397455, :]

X_test = X[1397455:1746818, :]
y_test = y[1397455:1746818, :]

In [8]:
def prepare_sequences(x_train, y_train, window_length):
    windows = []
    windows_y = []
    for i, sequence in enumerate(x_train):
        len_seq = len(sequence)
        for window_start in range(0, len_seq - window_length + 1):
            window_end = window_start + window_length
            window = sequence[window_start:window_end]
            windows.append(window)
            windows_y.append(y_train[i])
    return np.array(windows), np.array(windows_y)

In [9]:
X_train_new, y_train_new = prepare_sequences(X_train, y_train, 10)

In [10]:
X_train_new.shape, y_train_new.shape, X_train.shape

((2794910, 10), (2794910, 2), (1397455, 11))

In [11]:
X_train_new = X_train_new.reshape(X_train_new.shape[0], X_train_new.shape[1], 1)
np.sum(y_train_new, axis=0)

array([ 2025202.,   769708.])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

max_len = X_train_new.shape[1]
batch_size = 1000000#X_train_new.shape[0]

print('Building STATELESS model...')
model = Sequential()
model.add(LSTM(10, input_shape=(max_len, 1), return_sequences = False, stateful = False))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

model.fit(X_train_new, y_train_new,
          batch_size = batch_size,
          nb_epoch = 15,
          verbose = 1,
          validation_split = 0.1,
          shuffle = False,
          class_weight = {0: 1, 1: 2.7})

#score, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)

Building STATELESS model...
Train on 2515419 samples, validate on 279491 samples
Epoch 1/15


In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose = 1)
print("Accuracy: %.2f%%" % (scores[1]*100))

## regression

In [ ]:
df2 = df.drop(['day', 'pid', 'lineID', "click", "basket", "order", "number", "group", "pharmForm", "unit", 'campaignIndex'], axis=1)
print(df2.shape)

In [ ]:
X_r = np.array(df2.drop(["revenue"], axis=1))
y_r = np.array(df2[["revenue"]])

In [ ]:
X_trainr = X_r[0:1397455, :]
y_trainr = y_r[0:1397455, :]

X_testr = X_r[1397455:1746818, :]
y_testr = y_r[1397455:1746818, :]

In [ ]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(14, input_dim=11, init='normal', activation='relu'))
    model.add(Dense(7, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    return model

clf = KerasRegressor(build_fn = baseline_model, nb_epoch = 5, batch_size = 10000, verbose = 2)
kfold = KFold(n_splits = 10, random_state = 100)
results = cross_val_score(clf, np.array(X_trainr), np.array(y_trainr), cv = kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))


In [ ]:
def training(model, y_train):
    model.fit(X_trainr, y_train)
    score = cross_val_score(model, X_trainr, y_train, cv = 10)
    print('cross_val_score : %.2f' % (np.sum(score) / 10))

def testing_regression(model, y_test):
    pred = model.predict(X_testr).reshape(y_test.shape[0], 1)  
    y_test = y_test.reshape(X_testr.shape[0], 1)
    
    score = np.dot((pred - y_test).T, (pred - y_test))
    from sklearn.metrics import mean_squared_error
    score1 = mean_squared_error(y_test, pred, sample_weight = None, multioutput = 'uniform_average')
    print('===Euclidean_Distance===')
    print(score[0])
    print('===MSE===')
    print(score1)
    return pred

def plot_regression(test, pred):
    import matplotlib.pyplot as plt
    
    test = test.reshape(pred.shape[0],1)
    a = np.append(pred, test, axis=1)
    b = pd.DataFrame(a)
    b.columns = ["pred", "test"]  
    b.plot.scatter(x = 'test', y = 'pred', s = 50, alpha = 0.8)
    plt.title("scatter plot of pridiction and true")
    plt.show()

In [ ]:
training(clf, y_trainr)
pre_nn = testing_regression(clf, y_testr)
plot_regression(y_testr, pre_nn)

In [ ]:
from keras.models import Sequential  
from keras.layers.core import Dense, Activation  
from keras.layers.recurrent import LSTM

model_reg = Sequential()
model_reg.add(Embedding(top_words, embedding_vecor_length, input_length = max_review_length))
model_reg.add(LSTM(100))
model_reg.add(Dense(1, activation = 'linear'))
model_reg.compile(loss="mean_squared_error", optimizer="rmsprop")
model_reg.summary()

In [ ]:
model_reg.fit(X_trainr, y_trainr,
              nb_epoch = 3,
              batch_size = 64,
              validation_split = 0.1)

In [ ]:
pre_rnn = testing_regression(model_reg, y_testr)
plot_regression(y_testr, pre_rnn)